# Batch Processing
## Goal
Use Spark to compute the Pearson correlation coefficients.
The task requires that for each timestamp between 2018-12-06 and 2023-03-31 to compute the correlation for each pair of sensors and (2) determine the top-5 pairs with the highest correlation.

## Common imports and Environment variables

In [2]:
from pyspark.sql import SparkSession
from itertools import combinations
from pyspark.sql.functions import col, count, sum, first, mean, when
from pyspark.sql.window import Window
from tqdm import tqdm
from pyspark import Row
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 ' + \
                                    '--conf spark.driver.memory=4g  pyspark-shell '

## Spark session creation

In [3]:
#Create a Spark session
spark = SparkSession.builder.master("local[*]").appName("pearson_coefficient_batch").getOrCreate()

# Get Spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

#Read csv and create a Spark dataframe
df = spark.read.option("header", True).csv("data.csv")

#Nicer way to show but takes little more time
df.toPandas()

23/05/07 20:51:26 WARN Utils: Your hostname, toldo-Victus resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
23/05/07 20:51:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 20:51:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/07 20:51:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


,_c0,Date,Time gap,Sensor,Count,Average speed
0,0,2018-12-06,1,CAT17,0.0,-1.0
1,1,2018-12-06,1,CB02411,0.0,-1.0
2,2,2018-12-06,1,CB1101,0.0,-1.0
3,3,2018-12-06,1,CB1142,0.0,-1.0
4,4,2018-12-06,1,CB1143,0.0,-1.0
...,...,...,...,...,...,...
2725051,2725051,2023-03-31,96,CJE181,1.0,4.0
2725052,2725052,2023-03-31,96,CJM90,10.0,17.0
2725053,2725053,2023-03-31,96,CLW239,0.0,-1.0
2725054,2725054,2023-03-31,96,COM205,2.0,13.0


The next cell is used to pivot the dataframe. The cell calculate a pivoted dataframe with a column for each sensor, and a list of all the combination of sensors.

In [4]:
# pivot the dataframe
pivot_df = df.groupBy("Date", "Time gap").pivot("Sensor").agg(first("Count").alias("Count"),
                                                              first("Average speed").alias("Average speed"))

#Sort by date and time gap
pivot_df = pivot_df.withColumn("Time gap", col("Time gap").cast("int")).sort("Date", "Time gap")

# Store all columns names
columns = pivot_df.columns

# Store only "count" columns and timestamp
count_columns = [c for c in columns if c.endswith("_Count")]
# Create all possible combinations of sensors
pairs = list(combinations(count_columns, 2))

# Keep timestamp
count_columns.append("Date")
count_columns.append("Time gap")

# Filter out the dataframe to only keep those
filter_df = pivot_df.select(count_columns)

#Nicer way to show but takes little more time
filter_df.toPandas()

,CAT17_Count,CB02411_Count,CB1101_Count,CB1142_Count,CB1143_Count,CB1599_Count,CB1699_Count,CB2105_Count,CEE016_Count,CEK049_Count,CEK18_Count,CEK31_Count,CEV011_Count,CJE181_Count,CJM90_Count,CLW239_Count,COM205_Count,CVT387_Count,Date,Time gap
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-06,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-06,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-06,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-06,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-06,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151387,4.0,5.0,15.0,7.0,5.0,1.0,1.0,9.0,6.0,10.0,2.0,4.0,3.0,0.0,20.0,0.0,0.0,0.0,2023-03-31,92
151388,3.0,5.0,14.0,2.0,6.0,0.0,1.0,7.0,2.0,5.0,2.0,1.0,1.0,1.0,14.0,0.0,2.0,0.0,2023-03-31,93
151389,1.0,1.0,17.0,5.0,5.0,0.0,1.0,7.0,3.0,7.0,3.0,4.0,0.0,0.0,12.0,0.0,3.0,1.0,2023-03-31,94
151390,1.0,5.0,10.0,3.0,6.0,2.0,4.0,13.0,0.0,4.0,1.0,2.0,0.0,0.0,16.0,0.0,0.0,0.0,2023-03-31,95


This method is used to calculate the Pearson coefficient between two sensors.

In [5]:
# Define the function to calculate Pearson correlation coefficient with time component
def pearson_corr_coeff_time(count_df, i, j):
    # Assuming i and j are the names of two columns
    col_i = col(i)
    col_j = col(j)

    # We define the window function to take in consideration all the data from the current line to the beginning
    w = Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
    # Compute the window function count for column i and j
    ci_count = count(i).over(w)
    cj_count = count(j).over(w)

    # Use a sub-query to calculate the various column needed for the calculation of the Pearson coefficient
    # We modify the name of the sensor column to i and j
    subquery = count_df.select('Date', 'Time gap', col_i.alias('i'), col_j.alias('j'),
                               ci_count.alias('ci_count'),
                               cj_count.alias('cj_count'))
    # We define columns that are the sum of the passage of bikes of the sensor i/j
    subquery = subquery.select('Date', 'Time gap', 'i', 'j', sum('i').over(w).alias('ci_total'),
                               sum('j').over(w).alias('cj_total'), 'ci_count', 'cj_count')
    # We define columns that are the mean of the sensor i/j
    subquery = subquery.select('Date', 'Time gap', 'i', 'j', 'ci_total', 'cj_total', 'ci_count', 'cj_count',
                               (col('ci_total') / col('ci_count')).alias('ci_mean'),
                               (col('cj_total') / col('cj_count')).alias('cj_mean'))
    # We define columns that are the numerator and the two part of the denominator of the Pearson coefficient formula
    subquery = subquery.select('Date', 'Time gap', 'i', 'j', 'ci_total', 'cj_total', 'ci_count', 'cj_count', 'ci_mean',
                               'cj_mean',
                               (sum((col('i') - col('ci_mean')) * (col('j') - col('cj_mean'))).over(w)).alias(
                                   'numerator'),
                               ((sum((col('i') - col('ci_mean')) ** 2).over(w)) ** 0.5).alias('den1'),
                               ((sum((col('j') - col('cj_mean')) ** 2).over(w)) ** 0.5).alias('den2'))

    # Compute the Pearson correlation coefficient for each date and time gap
    rij = subquery.select("Date", "Time gap",
                          when((col("den1") == 0) | (col("den2") == 0), 0).otherwise(
                              col("numerator") / (col("den1") * col("den2"))))

    # Renaming columns for clarity
    i = i.rstrip("_Count")
    j = j.rstrip("_Count")
    rij = rij.withColumnRenamed(rij.columns[2], i + "_" + j)

    return rij

**The following cells are distributed like this otherwise spark crashes if you try all in one go.**

Now we get the last value of each sensor pair.

In [11]:
rij_dict = {}
for pair in tqdm(pairs[:20]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:30<00:00,  1.51s/it]                                 


In [12]:
for pair in tqdm(pairs[20:40]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:28<00:00,  1.43s/it]                                 


In [13]:
for pair in tqdm(pairs[40:60]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:26<00:00,  1.33s/it]                                 


In [14]:
for pair in tqdm(pairs[60:80]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:25<00:00,  1.26s/it]                                 


In [15]:
for pair in tqdm(pairs[80:100]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:25<00:00,  1.27s/it]                                 


In [16]:
for pair in tqdm(pairs[100:120]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:28<00:00,  1.42s/it]                                 


In [17]:
for pair in tqdm(pairs[120:140]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 20/20 [00:28<00:00,  1.44s/it]                                 


In [18]:
for pair in tqdm(pairs[140:]):
    rij = pearson_corr_coeff_time(filter_df, pair[0], pair[1])
    last_row = spark.createDataFrame(rij.tail(1))
    for i in last_row.collect():
        rij_dict[last_row.columns[2]] = i[last_row.columns[2]]

100%|██████████| 13/13 [00:17<00:00,  1.34s/it]                                 


We create a spark Dataframe with all the values collected

In [19]:
last_rj_df = spark.createDataFrame(Row(rij_dict))
pdf = last_rj_df.toPandas()
pdf

,CAT17_CB02411,CAT17_CB1101,CAT17_CB1142,CAT17_CB1143,CAT17_CB1599,CAT17_CB1699,CAT17_CB2105,CAT17_CEE016,CAT17_CEK049,CAT17_CEK18,...,CJE181_CJM90,CJE181_CLW239,CJE181_COM205,CJE181_CVT387,CJM90_CLW239,CJM90_COM205,CJM90_CVT387,CLW239_COM205,CLW239_CVT387,COM205_CVT387
0,0.630942,0.166352,0.605473,0.555556,0.649392,0.726944,0.679482,0.666109,0.69159,0.446886,...,0.628205,0.61882,0.46584,0.701515,0.693832,0.643247,0.679343,0.659451,0.663279,0.519127


Transpose the Dataframe taking only the top 5 row, so is more readable

In [20]:
pdf.T.nlargest(5, 0)

,0
CB2105_CJM90,0.835246
CB2105_CEK049,0.817282
CEK049_CJM90,0.813338
CB1599_CEK049,0.808896
CAT17_CVT387,0.798508
